In [ ]:
# Mic Recorder

In [18]:
import pyaudio
import wave
from google.cloud import speech
import io
import threading
import tempfile
import os

# Initialize the Speech-to-Text client
client = speech.SpeechClient()

# Function to record audio from the microphone
def record_audio(temp_file_path):
    # Setup parameters for the audio recording
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    sample_rate = 24000  # Record at 24kHz
    
    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print("Recording... Press Enter to stop.")

    # Open a new stream for recording
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=sample_rate,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

    # Function to read audio data in a loop
    def read_audio():
        while not stop_recording.is_set():
            data = stream.read(chunk)
            frames.append(data)

    # Create a threading event to stop recording
    stop_recording = threading.Event()

    # Start the audio recording thread
    recording_thread = threading.Thread(target=read_audio)
    recording_thread.start()

    # Wait for user input to stop recording
    input()
    stop_recording.set()
    recording_thread.join()

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    print("Recording complete.")

    # Save the recorded data as a WAV file
    with wave.open(temp_file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(sample_format))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(frames))

# Function to transcribe audio using Google Cloud Speech-to-Text
def transcribe_audio(temp_file_path):
    # Read the audio file
    with io.open(temp_file_path, "rb") as audio_file:
        content = audio_file.read()
    
    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=24000,
        language_code="ceb-PH",
        model="default",
        audio_channel_count=1,
        enable_word_confidence=True,
        enable_word_time_offsets=True,
    )

    # Detects speech in the audio file
    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=90)

    for result in response.results:
        print("Transcript: {}".format(result.alternatives[0].transcript))

# Main function
def main():
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
        temp_file_path = temp_file.name

    try:
        record_audio(temp_file_path)
        transcribe_audio(temp_file_path)
    finally:
        os.remove(temp_file_path)

if __name__ == "__main__":
    main()

Recording... Press Enter to stop.
Recording complete.
Waiting for operation to complete...
Transcript: hello Jose Carla call Uncle para marcar scholar Among Us with Deontay


In [19]:
# Import the Speech-to-Text client library
from google.cloud import speech

# Instantiates a client
client = speech.SpeechClient()

# The content of the audio file to transcribe
audio_content = "..."

# transcribe speech
audio = speech.RecognitionAudio(content=audio_content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=24000,
    language_code="ceb-PH",
    model="default",
    audio_channel_count=1,
    enable_word_confidence=True,
    enable_word_time_offsets=True,
)

# Detects speech in the audio file
operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=90)

for result in response.results:
  print("Transcript: {}".format(result.alternatives[0].transcript))

Waiting for operation to complete...
Transcript: LOL cozy car logos Apopka open now
